# advent of code 2024 - [day 20](https://adventofcode.com/2024/day/20)


### Parsing

In [2]:
def gen_lists(file='input.txt'):
    file = open(file, 'r')
    for ix, line in enumerate(file):
        for jx, c in enumerate(line.strip()):
            yield ix, jx, c

In [1]:
filename = "input.txt"
#filename = "test.txt"

## Graph Solution

In [2]:
import os

NEO4J_URI = os.environ['NEO4J_URI']
NEO4J_USERNAME = os.environ['NEO4J_USERNAME']
NEO4J_PASSWORD = os.environ['NEO4J_PASSWORD']

from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

from neo4j import GraphDatabase
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

/Users/pierrehalftermeyer/Documents/Events/AoC_2024/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/pierrehalftermeyer/Documents/Events/AoC_2024/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


2.13.0


In [5]:
# cleaning
def clean():
    queries = [
    "CALL apoc.schema.assert({},{});",
    """MATCH (n)
    CALL {WITH n DETACH DELETE n}
    IN TRANSACTIONS OF 1000 ROWS;""",
    """CALL gds.graph.list()
    YIELD graphName
    WITH graphName AS g
    CALL (g) {CALL gds.graph.drop(g) YIELD graphName RETURN graphName}
    WITH graphName RETURN graphName;"""]

    for q in queries:
        gds.run_cypher(q, {})

### Ingestion


In [6]:
def ingest(filename):

    tiles = [{'row':ix, 'col':jx, 'val':c} for ix, jx, c in list(gen_lists(filename))]
    
    clean()

    gds.run_cypher('CREATE INDEX tile_col_row IF NOT EXISTS FOR (t:Tile) ON (t.col, t.row)')
    gds.run_cypher('CREATE INDEX tile_val IF NOT EXISTS FOR (t:Tile) ON (t.val)')
    
    query_ingest = """
    UNWIND $tiles AS tile
    CREATE (:Tile {row:tile.row, col:tile.col, val:tile.val} )
    """

    gds.run_cypher(query_ingest, {"tiles":tiles})

## part 1

In [11]:
part1_build_queries = ["""MATCH (t:Tile)
WITH t.row AS row_num, t ORDER BY t.col
WITH row_num, collect(t) AS row
CALL apoc.nodes.link(row, 'EAST')""",
"""MATCH (t:Tile)
WITH t.col AS col_num, t ORDER BY t.row
WITH col_num, collect(t) AS col
CALL apoc.nodes.link(col, 'SOUTH')""",
"""
MATCH (a)-[r:SOUTH|EAST]->(b)
SET r.no_cheat=false
""",
"""
MATCH (a)-[r:SOUTH|EAST]->(b)
WHERE a.val IN ['S', 'E', '.'] AND b.val IN ['S', 'E', '.']
SET r.no_cheat=true, r.cost=1
""",
"""
MATCH (t:Tile) SET t.X_viz = t.col, t.Y_viz= -t.row 
""",
"""MATCH (t:Tile {val:'S'}) SET t:Start""",
"""MATCH (t:Tile {val:'E'}) SET t:End""",
"""
MATCH (t1:Tile WHERE t1.val IN ['S', 'E', '.'])-[:EAST|SOUTH]-(rock:Tile {val: '#'})-[:EAST|SOUTH]-(t2:Tile WHERE t2.val IN ['S', 'E', '.'])
WHERE t1 < t2
MERGE (t1)-[:GLITCH {no_cheat:false, cost:2}]->(t2)
"""
]

In [8]:
ingest(filename)
for q in part1_build_queries:
    gds.run_cypher(q)

## Part 1

In [5]:
gds.run_cypher("""MATCH (source)-[r:EAST|SOUTH {no_cheat:true}]-(target)
RETURN gds.graph.project(
  'maze',
  source,
  target,
  {}
)""")

,"gds.graph.project(\n 'maze',\n source,\n target,\n {}\n)"
0,"{'relationshipCount': 18864, 'graphName': 'maz..."


In [6]:
no_cheat_distance = gds.run_cypher("""MATCH (start:Start), (end:End)
CALL gds.shortestPath.dijkstra.stream('maze', {
    sourceNode: start,
    targetNodes: end
})
YIELD totalCost
RETURN
    toInteger(totalCost) AS no_cheat_distance""")['no_cheat_distance'][0]




In [7]:
no_cheat_distance

9432

In [3]:
gds.run_cypher('CREATE INDEX glitch_id IF NOT EXISTS FOR (gl:Glitch) ON (gl.id)')
gds.run_cypher('CREATE INDEX glitch_cost IF NOT EXISTS FOR (gl:Glitch) ON (gl.cost)')

""


In [ ]:
gds.run_cypher("""
MATCH (a)-[g:GLITCH]-(b)
WHERE NOT EXISTS {(glitch:Glitch {id:'maze_'+elementId(a)+'_'+elementId(g)+'_'+elementId(b)})} 
CALL (a, g, b) {
  CALL (a, g, b) {
    MATCH (source)-[r:EAST|SOUTH {no_cheat:true}]-(target)
    RETURN source, target, r
    
    UNION

    RETURN a AS source, b AS target, g AS r
  }
  WITH a, g, b, gds.graph.project(
    'maze_'+elementId(a)+'_'+elementId(g)+'_'+elementId(b),
    source,
    target,
    { relationshipProperties: r { .cost } }
  ) AS graph
  MATCH (start:Start), (end:End)
  CALL gds.shortestPath.dijkstra.stream('maze_'+elementId(a)+'_'+elementId(g)+'_'+elementId(b), {
      sourceNode: start,
      targetNodes: end,
    relationshipWeightProperty: 'cost'
  })
  YIELD totalCost
  WITH a, g, b, totalCost
  CALL gds.graph.drop('maze_'+elementId(a)+'_'+elementId(g)+'_'+elementId(b)) YIELD graphName
  WITH a, g, b, totalCost       
  MERGE (glitch:Glitch {id:'maze_'+elementId(a)+'_'+elementId(g)+'_'+elementId(b)})
      SET glitch.score = totalCost
} IN CONCURRENT TRANSACTIONS OF 10 ROWS
""")

In [12]:
gds.run_cypher("""
RETURN toInteger(COUNT{(g:Glitch WHERE g.score <= $no_cheat_distance - $threshold)}) AS part1
""",{'no_cheat_distance':no_cheat_distance, 'threshold':100})

,part1
0,1445
